# RAG

## Requirements

In [1]:
%%capture
!pip install transformers accelerate bitsandbytes langchain langchain-community sentence-transformers faiss-gpu pandas gdown
!pip install accelerate

## Dataset

In [2]:
!gdown --fuzzy https://drive.google.com/file/d/1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI
From (redirected): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI&confirm=t&uuid=35753983-ba30-4acc-985a-6a283477bcb1
To: /kaggle/working/IMDB_crawled.json
100%|█████████████████████████████████████████| 292M/292M [00:01<00:00, 171MB/s]


## Config

In [3]:
class Config:
    EMBEDDING_MODEL_NAME="thenlper/gte-base"
    LLM_MODEL_NAME="HuggingFaceH4/zephyr-7b-beta"
    K = 5 # top K retrieval

## Preprocessing

In [4]:
import pandas as pd

df = pd.read_json('IMDB_crawled.json')

In [5]:
df.head()

,id,title,first_page_summary,release_year,mpaa,budget,gross_worldwide,rating,directors,writers,stars,related_links,languages,countries_of_origin,summaries,synposis,reviews,genres
0,tt0071562,The Godfather Part II,The early life and career of Vito Corleone in ...,1974,R,"$13,000,000 (estimated)","$47,962,683",9.0,[Francis Ford Coppola],None,"[Al Pacino, Robert De Niro, Robert Duvall]",[https://imdb.com/title/tt0068646/?ref_=tt_sim...,"[English, Italian, Spanish, Latin, Sicilian]",[United States],[The early life and career of Vito Corleone in...,[The Godfather Part II presents two parallel s...,"[[Coppola's masterpiece is rivaled only by ""Th...","[Crime, Drama]"
1,tt0120737,The Lord of the Rings: The Fellowship of the Ring,A meek Hobbit from the Shire and eight compani...,2001,PG-13,"$93,000,000 (estimated)","$884,041,698",8.9,[Peter Jackson],None,"[Elijah Wood, Ian McKellen, Orlando Bloom]",[https://imdb.com/title/tt0167261/?ref_=tt_sim...,"[English, Sindarin]","[New Zealand, United States]",[A meek Hobbit from the Shire and eight compan...,[Galadriel (Cate Blanchett) (The Elven co-rule...,"[[Here is one film that lived up to its hype, ...","[Action, Adventure, Drama]"
2,tt0110912,Pulp Fiction,"The lives of two mob hitmen, a boxer, a gangst...",1994,R,"$8,000,000 (estimated)","$213,928,762",8.9,[Quentin Tarantino],None,"[John Travolta, Uma Thurman, Samuel L. Jackson]",[https://imdb.com/title/tt0137523/?ref_=tt_sim...,"[English, Spanish, French]",[United States],"[The lives of two mob hitmen, a boxer, a gangs...",[Narrative structure\nPulp Fiction's narrative...,[[I like the bit with the cheeseburger. It mak...,"[Crime, Drama]"
3,tt0068646,The Godfather,The aging patriarch of an organized crime dyna...,1972,R,"$6,000,000 (estimated)","$250,342,030",9.2,[Francis Ford Coppola],None,"[Marlon Brando, Al Pacino, James Caan]",[https://imdb.com/title/tt0071562/?ref_=tt_sim...,"[English, Italian, Latin]",[United States],[The aging patriarch of an organized crime dyn...,"[In late summer 1945, guests are gathered for ...",[['The Godfather' is the pinnacle of flawless ...,"[Crime, Drama]"
4,tt0111161,The Shawshank Redemption,"Over the course of several years, two convicts...",1994,R,"$25,000,000 (estimated)","$28,904,232",9.3,[Frank Darabont],"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",[https://imdb.com/title/tt0468569/?ref_=tt_sim...,[English],[United States],"[Over the course of several years, two convict...","[In 1947, Andy Dufresne (Tim Robbins), a banke...",[[The Shawshank Redemption is written and dire...,[Drama]


In [6]:
import os

os.makedirs('data', exist_ok=True)

# preprocess your data and only store the needed data as the context window for embedding model is limited
df = df[['first_page_summary', 'title', 'genres', 'rating']]
df.to_csv('data/imdb.csv', index=False)

## Vectorizer

load the CSV file and vectorize the rows using HuggingFaceEmbeddings.
Store the results using FAISS vectorstore.
Save the vectorestore in a pickle file for future usages.

In [7]:
import pickle

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores.utils import DistanceStrategy
from langchain.vectorstores.faiss import FAISS

from langchain_community.embeddings import HuggingFaceEmbeddings

# load the csv
csv_loader = CSVLoader('/kaggle/working/data/imdb.csv', encoding='utf-8')
documents = csv_loader.load()

# load the embeddings model
embedding_model = HuggingFaceEmbeddings()

# save embed the documents using the model in a vectorstore
vectorstore = FAISS.from_documents(documents, embedding_model, distance_strategy=DistanceStrategy.COSINE)

with open("data/vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-26 09:39:39.243947: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 09:39:39.244080: E external/local_x

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

load the vectorstore as a retriever.

In [8]:
with open("data/vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

# load the retriever from the vectorstore
retriever = vectorstore.as_retriever()

## LLM

load the quantized LLM.

In [9]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# load the quantization config
bnb_config = BitsAndBytesConfig()

model = AutoModelForCausalLM.from_pretrained(Config.LLM_MODEL_NAME, quantization_config=bnb_config, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained(Config.LLM_MODEL_NAME)

# init the pipeline
READER_LLM = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)

llm = HuggingFacePipeline(
    pipeline=READER_LLM,
)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


initialize the prompt template for the query chain. query chain is used to get a query from the chat history. you may change the prompt as you like to get better results.

In [21]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

class LoggerStrOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        # process the LLM output
        print(f"QUERY: {text}")
        return text

query_transform_prompt = PromptTemplate(
    input_variables=["messages"],
    template="""
"{messages}"
Please generate a search query for llm engine for the movie in above text. Just give one query as the output and the query should not be more than 20 words.
""" + "|SEP|"
)

# init the query chain
query_transforming_retriever_chain = (
    {"messages": RunnablePassthrough()}
    | query_transform_prompt
    | llm
    | StrOutputParser()
)

initialize the main retrieval chain that gives the resulting documents to LLM and gets the output back.

In [22]:
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate(
    input_variables=["context", "messages"],
    template="""You are a helpful assistant tasked with making recommendations and answering questions. 

Context - Movies Available for Selection:
{context}

--------------------------------------------------

User Queries:
{messages}

--------------------------------------------------

Based on the movies listed above and the user queries, generate a response that includes just one movie recommendation. Ensure the response is relevant and concise. Give the answer in this style:

Title: [the title of the movie (The year the movie was made)]

Genres: [the genres of the movie]

rating: [the rating of the movie]

just one move recommendation in this format.
""" + "|SEP|"
)

# init the retriver chain
retrieval_chain = (
    {"context" : retriever, "messages": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

write the conversation helper class for easier testing.

In [34]:
class Conversation:
    def __init__(self):
        self.messages = []
        self.retriever = retriever

    def add_assistant_message(self, message):
        self.messages.append(('assistant', message))

    def add_user_message(self, message):
        self.messages.append(('user', message))

    def get_messages(self):
        formatted_messages = []
        for role, message in self.messages:
            formatted_message = f"{role.capitalize()}: {message}"
            formatted_messages.append(formatted_message)
        return "\n--------------------------------------\n".join(formatted_messages)

    def chat(self, message):
        self.add_user_message(message)
        messages = self.get_messages()
        
        print(messages)
        print("*"*50)
        
        current_message = query_transforming_retriever_chain.invoke(messages).split("|SEP|")[-1]
        
        print("Current message: ", message)
        print("-"*50)
        print("Current Query: ", current_message)
        print("-"*50)
        
        response = retrieval_chain.invoke(current_message).split("|SEP|")[-1]
        self.add_assistant_message(' '.join(response.split("\n")[0:7]))
        
        return response

## Test

talk with the RAG to see how good it performs.

In [38]:
c = Conversation()
A = c.chat('give me a cool gangster movie')
print("Current Response: ")
print(A)

User: give me a cool gangster movie
**************************************************
Current message:  give me a cool gangster movie
--------------------------------------------------
Current Query:  >
"recommend a stylish and gritty crime drama with a charismatic antihero leading a dangerous gang in a neon-lit urban setting"
--------------------------------------------------
Current Response: 


Title: City of Industry (1993)

Genres: Crime, Drama, Thriller

rating: 6.1

This movie follows the story of a retired thief seeking revenge against the person who murdered his brother and partner, while also evading the law with the loot they stole. The movie is set in a neon-lit urban environment, and the main character, played by Robert De Niro, is a charismatic antihero leading a dangerous gang. The movie's stylish and gritty portrayal of crime and its urban setting make it a perfect fit for your query.


In [39]:
A = c.chat('give me a newer one')
print(A)

User: give me a cool gangster movie
--------------------------------------
Assistant:   Title: City of Industry (1993)  Genres: Crime, Drama, Thriller  rating: 6.1
--------------------------------------
User: give me a newer one
**************************************************
Current message:  give me a newer one
--------------------------------------------------
Current Query:  >
"Find a 2010s crime drama with a gritty style and a compelling protagonist who rises through the ranks of a criminal organization in a major city."
--------------------------------------------------


Title: Narcos (2015)

Genres: Crime, Drama, Biography

rating: 8.8

Narcos is a gritty crime drama series that follows the rise of the infamous drug lord Pablo Escobar and the DEA agent Steve Murphy's attempts to bring him down. However, in this recommendation, we're focusing on the character of Escobar's right-hand man, Juan Pablo Escobar, played by Pedro Pascal. The show's style is gritty and realistic, wit